In [84]:
import requests
from datetime import timezone, timedelta
import pandas as pd
import numpy as np

In [55]:
# we want to get a day's worth of BTC data from coingecko at a specified interval

CURRENCY = 'btc'

# use pre-acquired coin id data
ids = pd.read_csv("cg_coin_ids.csv")
coin_id = ids[ids['symbol'] == CURRENCY]['id'].values[0]

res = requests.get(f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=1')
res.raise_for_status()

coin_data = res.json()
unzipped = list(zip(*coin_data['prices']))
prices_raw = pd.DataFrame({'ms_datetime': unzipped[0], 'price': unzipped[1]})

In [119]:
# format dataset time
prices_raw['date_time'] = pd.to_datetime(prices_raw['ms_datetime'], unit='ms', utc=True)
prices = prices_raw[['date_time', 'price']].copy()
prices['local_dt'] = prices['date_time'].dt.tz_convert(timezone(-timedelta(hours=6))).dt.tz_convert(None)
prices = prices.sort_values(by='local_dt')[['local_dt', 'price']]

# add 15-minute intervals
prices['fifteen_min'] = np.tile(range(1, 4), prices.shape[0]//3 + 1)[:prices.shape[0]]
prices['fifteen_min'] = prices['fifteen_min'] == 1
m15 = prices[prices['fifteen_min']].reset_index(drop=True)

# add rsi

In [121]:
m15 = m15[['local_dt', 'price']]